## Práctica 03 &emsp;(23 de Septiembre)
## &emsp;Parte 02
## &emsp;Daniel Ricardo Ramírez Umaña, B45675

## Parte 02 <br>Cómo Usar Extracción de Variables en Datos Tabulares para Aprendizaje Automático

### &emsp;1. Lo aprendido:

#### &emsp; 1. Técnicas de Extracción de Variables para Preparación de Datos


Para preparar los datos de tal forma que estos puedan ser utilizados con algoritmos de *Machine Learning*, es necesario primero hacer un análisis de estos. Esto puede llegar a ser muy lento y requerir tanto conocimientos de Análisis de Datos como de algoritmos de Machine Learning. Para esto suele tratarse la **preparación de los datos de entrada** como un hiperparámetro, escogiendo los algoritmos de ML y los algoritmos de configuración apropiados. Sin embargo, a pesar que esto requiere menor expertiz, esto puede requerir una mucho más rendimiento computacional.

El **feature engineering** o **feature extraction** no permite encontrar un punto medio entre ambos casos, usando técnicas de preparación de datos y luego luego ajustar y evaluar un modelo en estos datos.


#### &emsp; 2. Conjunto de Datos y Línea Base de Rendimiento



#### &emsp;&emsp; 2.1. Dataset de Clasificación de Vino

* Contamos con un conjunto de datos para clasificación de vinos.
* Este cuenta con 13 variables de entrada.
* La variable de salida divide los vinos en 3 categorías.

In [1]:
# example of loading and summarizing the wine dataset
from pandas import read_csv

# ubicación del dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/wine.csv'

# carga el dataset como un data frame
df = read_csv(url, header=None)

# recuperar el arreglo numpy
data = df.values

# dividir las columnas en variables de entrada y de salida
X, y = data[:, :-1], data[:, -1]

# resumen de los datos cargados
print(X.shape, y.shape)

(178, 13) (178,)


#### &emsp;&emsp; 2.2 Rendimiento del Modelo Base

Como línea base en la clasificación de vinos usaremos el **modelo de Regresión Logística**

Como la biblioteca scikit-learn requiere requiere que las entradas sean numéricas y que la variable objetivo esté etiquetada, entonces esto será realizado.

&emsp;&emsp;Cargamos las dependencias

In [4]:
# baseline model performance on the wine dataset
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

&emsp;&emsp;Preparación mínima de los datos

In [23]:
# conjunto de datos mínimamente preparado
X = X.astype('float')
y = LabelEncoder().fit_transform(y.astype('str'))

&emsp;&emsp;Definimos nuestro modelo de predicción

In [24]:
# define the model
model = LogisticRegression(solver='liblinear')

&emsp;&emsp;Usando el modelo de validación cruzada estratificada k-fold repetida con 10 folds y 3 repeticiones y lo evaluaremos utilizando el ***accuracy*** de la clasificaión.

In [26]:
model = LogisticRegression(solver='liblinear')

# define el procedimiento de validación cruzada
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

# se evalua el modelo
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

# reporte de rendimiento
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.955 (0.049)


#### &emsp; 3. Enfoque de Extracción de Variables para Preparación de Datos

Para mejorar el rendimiento de nuestro modelo, es necesario usar el enfoque de extracción de variables para preparación de datos para lo que tenemos que seleccionar las técnicas de preparación más apropiadas.


Dependencias:

In [28]:
# data preparation as feature engineering for wine dataset
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

In [29]:
# transforms for the feature union
transforms = list()
transforms.append(('mms', MinMaxScaler()))
transforms.append(('ss', StandardScaler()))
transforms.append(('rs', RobustScaler()))
transforms.append(('qt', QuantileTransformer(n_quantiles=100, output_distribution='normal')))
transforms.append(('kbd', KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')))
transforms.append(('pca', PCA(n_components=7)))
transforms.append(('svd', TruncatedSVD(n_components=7)))

# create the feature union
fu = FeatureUnion(transforms)

Ahora uniremos las variables para entrenar nuestro modelo

In [30]:
# se define el modelo
model = LogisticRegression(solver='liblinear')

# se define el pipeline
steps = list()
steps.append(('fu', fu))
steps.append(('m', model))
pipeline = Pipeline(steps=steps)

In [31]:
# define the cross-validation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.968 (0.037)


Como se obsserva, el rendimiento de nuestro modelo, debido a la extracción de variables, logró subir su rendimiento a un 0.968.

Al reducir remover variables de entrada irrelevantes y redundantes podríamos mejorar el rendimiento del modelo, como en este caso en que usaremos RFE:

Requerimientos:

In [33]:
# data preparation as feature engineering with feature selection for wine dataset
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA

Definimos la delección de variables

In [36]:
# load the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/wine.csv'
df = read_csv(url, header=None)
data = df.values
X, y = data[:, :-1], data[:, -1]

# minimally prepare dataset
X = X.astype('float')
y = LabelEncoder().fit_transform(y.astype('str'))

# transforms for the feature union
transforms = list()
transforms.append(('mms', MinMaxScaler()))
transforms.append(('ss', StandardScaler()))
transforms.append(('rs', RobustScaler()))
transforms.append(('qt', QuantileTransformer(n_quantiles=100, output_distribution='normal')))
transforms.append(('kbd', KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')))
transforms.append(('pca', PCA(n_components=7)))
transforms.append(('svd', TruncatedSVD(n_components=7)))

# create the feature union
fu = FeatureUnion(transforms)

# define the feature selection
rfe = RFE(estimator=LogisticRegression(solver='liblinear'), n_features_to_select=15)

# define the model
model = LogisticRegression(solver='liblinear')

# define the pipeline
steps = list()
steps.append(('fu', fu))
steps.append(('rfe', rfe))
steps.append(('m', model))
pipeline = Pipeline(steps=steps)

# define the cross-validation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

# evaluate model
scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.989 (0.022)


Se puede ver una mejora en de desempeño del modelo al haber filtrado las variables poco relevantes y las reduntantes.

### &emsp; 2. Posible uso que le dará a esta materia como profesional.

* En mi futuro como profesional en Ciencias de la Computación, podré mejorar mis algoritmos aplicando las técnicas más adecuadas de forma estructurada con el objetivo de llegar al modelo más simple, pero a la vez preciso y eficiente posible y esto será gracias a la reducción de dimensionalidad de mis conjuntos de datos y al filtrar aquellas variables que me podrían generar problemas en el entrenamiento al introducir ruido.

### &emsp; 3. Aspectos que no le quedan claros, o que quisiera conocer mejor.

* No me quedó muy claro exactamente qué es lo que hacen las funciones `MinMaxScaler()`, `StandardScaler()` y `RobustScaler()`, por lo que me daré a la tarea de investigar más al respecto en documentación complementaria.
